In [ ]:
import requests

def get_location_by_coordinates(lat, lon, api_key, limit=1):
    url = f"http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit={limit}&appid={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        location_data = response.json()

        if location_data:
            return location_data  # Return the first result if available
        else:
            return "No location data found for the given coordinates."
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"
    except Exception as err:
        return f"An error occurred: {err}"

# Input latitude, longitude, and your API key
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))
api_key = "7e24b2a1b24a409661399c33c48e0993"

location_info = get_location_by_coordinates(latitude, longitude, api_key)
print(location_info)


Enter latitude: 25.7464
Enter longitude: 82.6837
[{'name': 'Jaunpur', 'local_names': {'hi': 'जौनपुर', 'fr': 'Jawnpur', 'pa': 'ਜੌਨਪੁਰ', 'ar': 'جونبور', 'bn': 'জৌনপুর', 'kn': 'ಜೌನಪುರ', 'ml': 'ജൗൻപൂർ', 'tr': 'Jaunpûr', 'ja': 'ジャウンプル'}, 'lat': 25.7472, 'lon': 82.689, 'country': 'IN', 'state': 'Uttar Pradesh'}]


In [ ]:
import requests

def get_district_by_coordinates(lat, lon, api_key, limit=1):
    url = f"http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit={limit}&appid={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        location_data = response.json()

        if location_data:
            # Extract district or place name (may vary depending on the API response)
            district = location_data[0].get("name", "District name not found")
            state = location_data[0].get("state", "State not found")
            country = location_data[0].get("country", "Country not found")
            return f"District: {district}, State: {state}, Country: {country}"
        else:
            return "No location data found for the given coordinates."
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"
    except Exception as err:
        return f"An error occurred: {err}"

# Input latitude, longitude, and your API key
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))
api_key = "7e24b2a1b24a409661399c33c48e0993"  # Replace with your actual OpenWeatherMap API key

district_info = get_district_by_coordinates(latitude, longitude, api_key)
print(district_info)


Enter latitude: 25.4550626
Enter longitude: 81.8349598
District: Prayagraj, State: Uttar Pradesh, Country: IN


In [ ]:
import requests

def get_district_by_coordinates(lat, lon, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}+{lon}&key={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        location_data = response.json()

        if location_data and location_data.get("results"):
          # print(location_data)
          print(location_data["results"][0]["components"]["road"])
          print(location_data["results"][0]["components"]["suburb"])
          print(location_data["results"][0]["components"]["state_district"])
        else:
            return "No location data found for the given coordinates."
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"
    except Exception as err:
        return f"An error occurred: {err}"

# Input latitude, longitude, and your API key
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))
api_key = "209fd3b3c460499e91f4fc86e110513b"  # Replace with your actual OpenCage API key

district_info = get_district_by_coordinates(latitude, longitude, api_key)
# print(district_info)


Enter latitude: 25.4550626
Enter longitude: 81.8349598
Sardar Patel Marg
Civil Lines
Prayagraj
None


In [ ]:
import requests
import pandas as pd

def get_location_data(lat, lon, api_key):
    """
    Fetches the location data using the OpenCage API and returns road, suburb, and state district.
    """
    url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}+{lon}&key={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        location_data = response.json()

        if location_data and location_data.get("results"):
            road = location_data["results"][0]["components"].get("road", "")
            suburb = location_data["results"][0]["components"].get("suburb", "")
            state_district = location_data["results"][0]["components"].get("state_district", "")
            return road, suburb, state_district
        else:
            return "", "", ""
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return "", "", ""
    except Exception as err:
        print(f"An error occurred: {err}")
        return "", "", ""

# List of API keys to use
api_keys = [
    "209fd3b3c460499e91f4fc86e110513b",  # Replace with your actual keys
    "b0c319e774e24be18d742768cdb5877f",
    # Add more keys if available
]

# Input files and settings
input_file = 'combined_amenities.csv'
output_file = 'updated_combined_amenities.csv'
api_key_index = 0
requests_count = 0
max_requests_per_key = 2400

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Create new columns for road, suburb, and state_district
df['road'] = ''
df['suburb'] = ''
df['state_district'] = ''

# Iterate through each row in the DataFrame to fetch the location data
for index, row in df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']

    # Get the current API key
    current_api_key = api_keys[api_key_index]

    # Get location details from the API
    road, suburb, state_district = get_location_data(lat, lon, current_api_key)

    # Update the DataFrame with the fetched data
    df.at[index, 'road'] = road
    df.at[index, 'suburb'] = suburb
    df.at[index, 'state_district'] = state_district

    # Update the request count
    requests_count += 1

    # Check if we need to switch the API key
    if requests_count >= max_requests_per_key:
        requests_count = 0  # Reset the count
        api_key_index = (api_key_index + 1) % len(api_keys)  # Move to the next API key
        print(f"Switching to API key: {api_keys[api_key_index]}")

    print(f"Processed {index+1}/{len(df)}: Road={road}, Suburb={suburb}, State District={state_district}")

# Write the updated data back to a new CSV file
df.to_csv(output_file, index=False)
print(f"Updated data saved to {output_file}")


Processed 1/4177: Road=shiv temple, Suburb=, State District=Azamgarh
Processed 2/4177: Road=Naini Bridge, Suburb=New Bairahana, State District=Prayagraj
Processed 3/4177: Road=Luxa Road, Suburb=, State District=Varanasi
Processed 4/4177: Road=Luxa Road, Suburb=, State District=Varanasi
Processed 5/4177: Road=The Mall Road, Suburb=Cantonment, State District=Varanasi
Processed 6/4177: Road=Raja Sir Motichand Road, Suburb=, State District=Varanasi
Processed 7/4177: Road=Vidyapeeth Lanka Road, Suburb=Cantonment, State District=Varanasi
Processed 8/4177: Road=Hashimpur Road, Suburb=George Town, State District=Prayagraj
Processed 9/4177: Road=Jawaharlal Nehru Road, Suburb=George Town, State District=Prayagraj
Processed 10/4177: Road=Hashimpur Road, Suburb=George Town, State District=Prayagraj
Processed 11/4177: Road=Hamilton Road, Suburb=George Town, State District=Prayagraj
Processed 12/4177: Road=Sardar Patel Marg, Suburb=Civil Lines, State District=Prayagraj
Processed 13/4177: Road=Semi C

In [ ]:
import pandas as pd

# Open the CSV file
input_file = 'updated_combined_amenities.csv'
df = pd.read_csv(input_file)

# Create a dictionary counting the occurrences of each unique road value
road_counts = df['road'].value_counts().to_dict()

# Add a new column 'road_count' to the DataFrame using the dictionary
df['road_count'] = df['road'].map(road_counts)

# Save the updated DataFrame to a new CSV file
output_file = 'final_combined_amenities.csv'
df.to_csv(output_file, index=False)

print(f"Updated data with road count saved to {output_file}")


Updated data with road count saved to final_combined_amenities.csv


In [ ]:
import pandas as pd

# Open the CSV file
input_file = 'updated_combined_amenities.csv'
df = pd.read_csv(input_file)

# Create dictionaries counting the occurrences of each unique value in 'road', 'suburb', and 'state_district'
road_counts = df['road'].value_counts().to_dict()
suburb_counts = df['suburb'].value_counts().to_dict()
district_counts = df['state_district'].value_counts().to_dict()

# Define a function to map values with a default of 0 for unknowns
def map_with_default(value, count_dict):
    return count_dict.get(value, 0)

# Add new columns 'road_count', 'suburb_count', and 'district_count' to the DataFrame
df['road_count'] = df['road'].map(lambda x: map_with_default(x, road_counts))
df['suburb_count'] = df['suburb'].map(lambda x: map_with_default(x, suburb_counts))
df['district_count'] = df['state_district'].map(lambda x: map_with_default(x, district_counts))

# Save the updated DataFrame to a new CSV file
output_file = 'final_combined_amenities.csv'
df.to_csv(output_file, index=False)

print(f"Updated data with road, suburb, and district counts saved to {output_file}")


Updated data with road, suburb, and district counts saved to final_combined_amenities.csv
